<a href="https://colab.research.google.com/github/Dhyanesh-Panchal/GSR-research-and-stress-detection/blob/master/DNN_model_with_only_phasicGSR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

### Prepare the the Data

In [ ]:
data_sub1 = pd.read_csv("https://raw.githubusercontent.com/Dhyanesh-Panchal/GSR-research-and-stress-detection/master/preprocessed_data/sub_1.csv")
data_sub1.head()

,time,gsr,video,valence,arousal,EDA_Clean,EDA_Phasic,EDA_Tonic
0,0,11.749,10,5.0,5.0,11.749013,-0.060830,11.809843
1,50,11.729,10,5.0,5.0,11.738831,-0.071808,11.810639
2,100,11.737,10,5.0,5.0,11.729231,-0.082219,11.811450
3,150,11.713,10,5.0,5.0,11.720394,-0.091883,11.812276
4,200,11.717,10,5.0,5.0,11.712337,-0.100781,11.813118


In [ ]:
# windowing function
def rolling_window(df,window_size):
    windows = []
    for i in range(len(df)-window_size+1):
        single_window = df.iloc[i:i+window_size]
        # print(single_window)
        windows.append(single_window)
    return windows
windowed_data = rolling_window(data_sub1,100)
intermediate_array = []
for frame in windowed_data:
    intermediate_array.append(list(frame["EDA_Phasic"]))

phasic_windowed = pd.DataFrame(np.array(intermediate_array))
phasic_windowed.shape

(48933, 100)

In [ ]:
X = np.array(phasic_windowed).reshape(len(phasic_windowed),100,1)

In [ ]:
y = np.array(pd.read_csv('https://raw.githubusercontent.com/Dhyanesh-Panchal/GSR-research-and-stress-detection/master/labels.csv'))
y.shape

(48933, 2)

In [ ]:
# Create a CNN model.
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(32, 5, activation='relu', input_shape=(100, 1)),
  tf.keras.layers.MaxPooling1D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1)
])

# Compile the CNN model.
model.compile(loss='mse', optimizer='adam')

In [ ]:
model.fit(X, y, epochs=150)

Epoch 1/150
1530/1530 [==============================] - 5s 3ms/step - loss: 2.7766
Epoch 2/150
1530/1530 [==============================] - 6s 4ms/step - loss: 2.7765
Epoch 3/150
1530/1530 [==============================] - 5s 3ms/step - loss: 2.7717
Epoch 4/150
1530/1530 [==============================] - 5s 3ms/step - loss: 2.7718
Epoch 5/150
1530/1530 [==============================] - 6s 4ms/step - loss: 2.7719
Epoch 6/150
1530/1530 [==============================] - 5s 3ms/step - loss: 2.7701
Epoch 7/150
1530/1530 [==============================] - 5s 3ms/step - loss: 2.7711
Epoch 8/150
1530/1530 [==============================] - 5s 3ms/step - loss: 2.7684
Epoch 9/150
1530/1530 [==============================] - 5s 3ms/step - loss: 2.7663
Epoch 10/150
1530/1530 [==============================] - 5s 4ms/step - loss: 2.7661
Epoch 11/150
1530/1530 [==============================] - 5s 3ms/step - loss: 2.7641
Epoch 12/150
1530/1530 [==============================] - 5s 3ms/step - lo

In [ ]:
model.save('./model_150ep.h5',save_format='h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
